<div align="center"><img src="./images/DLI_Header.png"></div>

# Improving the Reduction Performance with `cub`

In this notebook we introduce the `cub` library and utilize it in the Jacobi iteration code to greatly improve the performance of the `jacobi` kernel. This notebook is a bit of an aside from the main focus of this course, however, the gains made from using highly optimized libraries like `cub` make it worth your being exposed to it.

## Objectives

By the time you complete this notebook you will:

- Be able to use the `cub` library to perform optimized block level reductions, greatly reducing atomic pressure on symmetric data being used by all GPUs.

## Significant Atomic Serialization

In the previous notebook, hundreds of thousands of threads made atomic writes to `l2_norm`:

```cpp
int main()
{
    ...

    float* d_l2_norm = (float*) nvshmem_malloc(sizeof(float));
    jacobi<<<blocks, threads_per_block>>>(f_old, f, d_l2_norm, N); // Grid launched with hundreds of thousands of threads.
}

__global__ void jacobi (const float* f_old, float* f, float* l2_norm, int N)
{
    ...

    float l2 = (f[idx] - f_old[idx]) * (f[idx] - f_old[idx]);
    // Here in the kernel, (practically) every thread is making atomic writes to symmetric `l2_norm`.
    atomicAdd(l2_norm, l2);
}
```

This means we have hundreds of thousands of threads performing an atomic reduction to the same variable. Knowing this we might propose that there is a great deal of atomic serialization in our program, which could have a significant impact on performance.

## Profile with Nsight Compute

[Nsight Compute](https://developer.nvidia.com/nsight-compute) is a kernel profiling tool, and here we will use its command line tool `ncu` to profile the solution code from the previous notebook.

`profile_one_jacobi_PE.sh` is a simple script that profiles only PE 0 (and skips the first few kernels to allow the GPU to warm up). Run the following cell to compile and run the solution application, generating profiling output for the first PE:

In [ ]:
!!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o jacobi_solution_step1 solutions/jacobi_step1.cpp
!nvshmrun -np $NUM_DEVICES ./code/profile_one_jacobi_PE.sh ./jacobi_solution_step1
!ncu -i report.ncu-rep

## Observations

What we see is that despite reasonable achieved occupancy (`Section:Occupancy` -> `Achieved Occupancy`), we are nowhere near peak theoretical memory bandwidth (`Section: GPU Speed Of Light` -> `Memory [%]`).

This supports our earlier hypothesis, and is likely because we have hundreds of thousands of threads performing an atomic reduction to the same variables.

## How to Reduce the Atomic Serialization

An approach to limit the amount of atomic serialization is to [perform as much of the reduction within the block as possible](https://developer.nvidia.com/blog/faster-parallel-reductions-kepler/). In such an approach, each block would efficiently reduce each of its threads' `l2` values, and then, have only one thread for each block, perform the atomic add to the symmetrical `l2_norm` variable.

## Block Level Reduction with `cub`

[cub](https://docs.nvidia.com/cuda/cub/index.html) is a header library provided by NVIDIA as part of CUDA[<sup>1</sup>](#footnote1) that provides interfaces for primitive operations that are often used in kernels such as reductions and scan operations.

For our current situation, let's use the [BlockReduce](https://nvlabs.github.io/cub/classcub_1_1_block_reduce.html) interface in `cub` to perform block-level reductions, and then only have thread `0` in each block perform the atomic add to the symmetric data `l2_norm`.

### Using `cub::BlockReduce`

To use `cub` we first add the header:

```cpp
#include <cub/cub.cuh>
```

To use `BlockReduce` we then define a `BlockReduce` type with as many threads per block as we want to use:

```cpp
typedef cub::BlockReduce<float, 256> BlockReduce;
```

`BlockReduce` will be utilizing shared memory to perform its efficient block-level reduction, so next we allocate shared memory for it to use:

```cpp
__shared__ typename BlockReduce::TempStorage temp_storage;
```

Finally we perform the reduction, in our case a sum reduction:

```cpp
float reduced_value = BlockReduce(temp_storage).Sum(value_to_reduce);
```

## Exercise: Use Block Reduce in Jacobi Code

For this exercise, look for the FIXMEs to refactor [exercises/jacobi_step2.cpp](exercises/jacobi_step2.cpp) (which starts from the solution of the last notebook) to perform block reduction with `cub`, only performing the atomic write to the symmetric `l2_norm` with from thread `0` in each block.

Check [solutions/jacobi_step2.cpp](solutions/jacobi_step2.cpp) if you need help.

In [ ]:
!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o jacobi_step2 exercises/jacobi_step2.cpp
!nvshmrun -np $NUM_DEVICES ./jacobi_step2

## Profile with Nsight Compute

Before proceeding, if you weren't able to complete the exercise on your own, replace `exercises/jacobi_step2.cpp` with `solutions/jacobi_step2.cpp` in the compilation cell above, and run the cell so that you have a working solution to profile below.

Let's verify with Nsight Compute that we achieve a significantly higher percentage of peak throughput.

In [ ]:
!nvshmrun -np $NUM_DEVICES ./code/profile_one_jacobi_PE.sh ./jacobi_step2
!ncu -i report.ncu-rep

## Observations

We note that `Section: GPU Speed Of Light` -> `Memory [%]` has greatly improved and that `Section: GPU Speed Of Light` -> `Elapsed Cycles ` has greatly decreased. Interestingly we see in `Section: Launch Statistics` -> `Static Shared Memory Per Block` the shared memory we allocated for use by `BlockReduce`'s efficient reduction.

## Next

The Jacobi iteration stencil is relatively simple, and we only looked at it in one dimension, but methods like this one are omnipresent in scientific computing. We can now see how straightforward it is to use NVSHMEM in problems that use a standard domain decomposition.

In the next and final notebook, we present you with a fully functional single GPU CUDA implementation of a 1D wave function solver, and ask you to combine all your skills thus far to refactor it using NVSHMEM.

Please open the next notebook: [_Final Exercise_](14_Wave.ipynb).

## Additional Resources

For a much deeper dive into Jacobi Iteration using a number of multi GPU techniques, please visit [github.com/NVIDIA/multi-gpu-programming-models](https://github.com/NVIDIA/multi-gpu-programming-models).

## Footnotes

<span id="footnote1">1</span>: Starting with CUDA 11.0, cub is available as a first-class citizen of the CUDA toolkit available directly in its `include/` directory. In prior versions of CUDA, cub was bundled as part of Thrust in the CUDA toolkit. In either case, it is [available on GitHub](https://github.com/NVIDIA/cub).